# Лабораторная работа 6

## Рекуррентные нейронные сети

1. Цель работы: Изучение и практическое применение рекуррентных нейронных сетей для анализа и моделирования временных рядов.

2. Основные задачи работы

    1. Изучение теоретических основ рекуррентных нейронных сетей.

    2. Разработка и обучение рекуррентной нейронной сети для анализа временных рядов.

    3. Применение обученной модели для прогнозирования временных рядов.

    4. Оценка и анализ полученных результатов.


# Выполнение работы

Задание 1. Задача генерации текста посимвольно с использованием RNN

В данной задаче входной последовательностью для RNN являются первые шесть символов текста, а целью является предсказание следующего (седьмого) символа. Затем предсказанный символ добавляется к входной последовательности, и процесс повторяется для генерации последующих символов.

 

Процесс рекуррентной генерации текста состоит из следующих шагов:

1. Подготовка обучающего набора: Обучающий набор представляет собой текстовые данные, которые могут быть скопированы или взяты из какого-либо источника. Весь текст разбивается на отдельные символы, и каждый символ становится элементом последовательности.

2. Предобработка данных: Символы текста преобразуются в числовой формат, например, с помощью one-hot encoding, где каждый символ представлен вектором размерности, равной общему количеству уникальных символов в обучающем наборе.

3. Создание RNN модели: Модель RNN создается, состоящая из рекуррентного слоя (RNN layer) и выходного слоя (output layer). Рекуррентный слой обрабатывает входную последовательность символов и передает информацию о предыдущем состоянии в следующий шаг. Выходной слой генерирует вероятности для следующего символа.

4. Обучение модели: RNN модель обучается на обучающем наборе с использованием метода обратного распространения ошибки. Происходит подбор оптимальных весов и параметров модели для минимизации ошибки предсказания следующего символа.

5. Генерация текста: После завершения обучения модели можно использовать для генерации нового текста. Процесс начинается с задания начальной последовательности из шести символов. Затем RNN модель принимает эту последовательность в качестве входа и предсказывает вероятности для следующего символа. Символ с наибольшей вероятностью выбирается и добавляется к текущей последовательности. После этого процесс повторяется для генерации следующего символа. Таким образом, можно генерировать текст, символ за символом, продолжая последовательность до достижения заданной длины или условия остановки.

6. Оценка результатов: Сгенерированный текст можно оценить с помощью различных метрик, таких как перплексия, сходство с исходным текстом или субъективное оценивание качества текста.

2. Подготовка набора данных временного ряда для обучения и тестирования модели.

In [49]:
import numpy as np

def read_file_to_string(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Загрузка текстовых данных
text = read_file_to_string("./biblefull.txt").lower()
# Создание уникального набора символов
chars = sorted(list(set(text)))
print(f"Уникальные символы: {chars}")

# Создание словаря для преобразования символов в индексы
char_to_index = {c: i for i, c in enumerate(chars)}
index_to_char = {i: c for i, c in enumerate(chars)}

# Параметры
seq_length = 12
step = 1

# Подготовка обучающих последовательностей
sequences = []
next_chars = []

for i in range(0, len(text) - seq_length, step):
    sequences.append(text[i: i + seq_length])
    next_chars.append(text[i + seq_length])

# Преобразование символов в one-hot encoding
X = np.zeros((len(sequences), seq_length, len(chars)), dtype=np.bool_)
y = np.zeros((len(sequences), len(chars)), dtype=np.bool_)

for i, seq in enumerate(sequences):
    for j, char in enumerate(seq):
        X[i, j, char_to_index[char]] = 1
    y[i, char_to_index[next_chars[i]]] = 1

Уникальные символы: ['\n', ' ', '!', "'", ',', '-', '.', ':', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


3. Разработка архитектуры рекуррентной нейронной сети с использованием фреймворка глубокого обучения, такого как TensorFlow.

In [50]:
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import Adam

# Создание улучшенной модели RNN
model = Sequential([
    layers.LSTM(256, return_sequences=True, input_shape=(seq_length, len(chars))),  # LSTM с возвратом последовательностей
    layers.Dropout(0.2),
    layers.LSTM(256),  # Второй LSTM слой
    layers.Dense(128, activation='relu'),  # Полносвязный слой
    layers.Dense(len(chars), activation='softmax')  # Выходной слой для предсказания символа
])

# Компиляция модели
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy')

4. Обучение разработанной модели на обучающем наборе данных с использованием алгоритма обратного распространения ошибки.

In [51]:
# Обучение модели
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
32271/32271 ━━━━━━━━━━━━━━━━━━━━ 1955s 61ms/step - loss: 1.4683
Epoch 2/10
32271/32271 ━━━━━━━━━━━━━━━━━━━━ 1952s 60ms/step - loss: 1.0891
Epoch 3/10
32271/32271 ━━━━━━━━━━━━━━━━━━━━ 2022s 63ms/step - loss: 1.0505
Epoch 4/10
32271/32271 ━━━━━━━━━━━━━━━━━━━━ 1982s 61ms/step - loss: 1.0296
Epoch 5/10
32271/32271 ━━━━━━━━━━━━━━━━━━━━ 2196s 68ms/step - loss: 1.0169
Epoch 6/10
32271/32271 ━━━━━━━━━━━━━━━━━━━━ 2129s 66ms/step - loss: 1.0058
Epoch 7/10
32271/32271 ━━━━━━━━━━━━━━━━━━━━ 2004s 62ms/step - loss: 0.9986
Epoch 8/10
32271/32271 ━━━━━━━━━━━━━━━━━━━━ 1979s 61ms/step - loss: 0.9918
Epoch 9/10
32271/32271 ━━━━━━━━━━━━━━━━━━━━ 2035s 63ms/step - loss: 0.9867
Epoch 10/10
32271/32271 ━━━━━━━━━━━━━━━━━━━━ 2083s 65ms/step - loss: 0.9809


5. Применение обученной модели для прогнозирования значений временного ряда на тестовом наборе данных.

In [52]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-10) / temperature  # Add a small value to avoid log(0)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)  # Normalize to get probabilities
    return np.random.choice(len(preds), p=preds)  # Sample from the probability distribution

def generate_text(model, start_string, num_generate=100, temperature=1.0):
    # Prepare the input for the model
    input_eval = np.zeros((1, seq_length, len(chars)))  # Create an empty array
    for i, char in enumerate(start_string):
        input_eval[0, i, char_to_index[char]] = 1

    generated_text = start_string

    for _ in range(num_generate):
        predictions = model.predict(input_eval)
        predicted_id = sample(predictions[-1], temperature)  # Sample using the new function

        generated_text += index_to_char[predicted_id]

        # Update input for the next iteration
        input_eval = np.roll(input_eval, -1, axis=1)  # Shift left
        input_eval[0, -1, predicted_id] = 1  # Add new character

    return generated_text

# Generating text with a specified temperature
generated_text = generate_text(model, "And God said".lower(), num_generate=100, temperature=0.2)
print(generated_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━

6. Анализ полученных результатов, выявление достоинств и недостатков модели.

Оценка результатов генерации текста RNN

1.	Перплексия: Перплексия (Perplexity) измеряет, насколько модель “смущена” при генерации текста. Чем ниже перплексия, тем лучше модель предсказывает последовательности символов, основываясь на обучающем наборе. Расчёт перплексии позволит количественно оценить качество предсказаний модели. В данном случае, скорее всего, перплексия будет высокой, так как модель генерирует текст без явного смысла и логической структуры.
2.	Сходство с исходным текстом: Сгенерированный текст имеет несколько признаков оригинального текста, такие как повторение некоторых символов и небольшие совпадения в структуре слов. Однако, отсутствует семантическая связь и логические фразы. Это указывает на то, что модель плохо запомнила шаблоны слов и фраз в исходных данных.
3.	Субъективное качество текста: На основе примера видно, что текст не несёт осмысленных фраз. Модель выучила только некоторые комбинации символов и слова частично, но ещё не понимает структуры предложений или правил языка, что делает текст бессмысленным.

Рекомендации для улучшения

- Увеличить объём данных: Для более осмысленных результатов модели требуется больше данных, поскольку текст из 10,000 строк недостаточен для улавливания паттернов длинных текстов, таких как Библия.
- Повысить количество эпох обучения: Дополнительные эпохи могут улучшить обучение модели и привести к более осмысленному тексту.
- Использовать более сложную архитектуру: Добавление слоёв или увеличение нейронов в RNN может помочь лучше захватывать контексты и зависимости между символами и словами.

Эти шаги помогут добиться большей осмысленности и когерентности сгенерированного текста.

### Контрольные вопросы

#### 1. Чем отличается рекуррентная нейронная сеть от обычной прямой нейронной сети?

Рекуррентные нейронные сети (РНС) и обычные прямые нейронные сети (или полносвязные нейронные сети) отличаются в основном своей архитектурой и предназначением. Вот ключевые отличия:

1. Архитектура

	- Прямые нейронные сети: Состоят из входного слоя, одного или нескольких скрытых слоев и выходного слоя. Данные передаются в одном направлении: от входного к выходному. Каждый нейрон в слое связан с каждым нейроном следующего слоя.
	- Рекуррентные нейронные сети: Имеют циклические соединения, что позволяет передавать информацию не только вперед, но и назад. Это означает, что состояние сети может зависеть от предыдущих состояний, что позволяет обрабатывать последовательные данные (например, временные ряды, текст).

2. Обработка данных

	- Прямые нейронные сети: Лучше подходят для обработки фиксированных входных данных, таких как изображения, где каждый элемент не зависит от предыдущих.
	- Рекуррентные нейронные сети: Предназначены для обработки последовательностей, где порядок и контекст данных имеют значение, например, в текстах, временных рядах или аудиосигналах.

3. Запоминание информации

	- Прямые нейронные сети: Запоминают информацию только через параметры (веса) сети, которые обучаются во время тренировки.
	- Рекуррентные нейронные сети: Могут сохранять информацию о предыдущих входах благодаря внутреннему состоянию (или скрытому состоянию), что позволяет им “помнить” предыдущие данные и учитывать их в текущих вычислениях.

4. Обучение

	- Прямые нейронные сети: Используют стандартные алгоритмы обратного распространения (backpropagation) для обучения.
	- Рекуррентные нейронные сети: Используют модифицированный алгоритм обратного распространения, известный как обратное распространение через время (Backpropagation Through Time, BPTT), для учета зависимостей во времени.

#### 2. Какие проблемы могут возникнуть при обучении рекуррентных нейронных сетей и как они могут быть решены?

Обучение рекуррентных нейронных сетей (РНС) может столкнуться с несколькими проблемами, связанными с особенностями обработки последовательных данных. Вот основные из них и возможные решения:

1. Проблема исчезающего градиента

	- Описание: При обучении РНС, особенно на длинных последовательностях, градиенты, которые используются для обновления весов, могут стремиться к нулю. Это приводит к тому, что обновления весов становятся слишком малыми, и сеть не может учиться эффективно.
	- Решение:
        - Использование LSTM (Long Short-Term Memory) и GRU (Gated Recurrent Units): Эти архитектуры содержат специальные механизмы (гейты), которые позволяют лучше сохранять долгосрочные зависимости и предотвращают исчезновение градиента.
        - Инициализация весов: Применение более подходящей инициализации весов может помочь смягчить проблему.
        - Регуляризация: Использование методов регуляризации, таких как L2-регуляризация, может помочь уменьшить влияние исчезающих градиентов.

2. Проблема взрывного градиента

	- Описание: Противоположная проблема, при которой градиенты становятся слишком большими, что может привести к нестабильности во время обучения и переполнению.
	- Решение:
        - Ограничение градиента (Gradient Clipping): Установка порога для градиентов, чтобы избежать их слишком большого значения, что помогает контролировать обновления весов.
        - Регуляризация: Аналогично, регуляризация может помочь предотвратить переполнение.

3. Долгосрочные зависимости

	- Описание: РНС могут плохо справляться с задачами, где важны зависимости на больших расстояниях в последовательности. Это может привести к потере информации.
	- Решение:
        - Использование LSTM и GRU: Эти архитектуры специально разработаны для обработки долгосрочных зависимостей.
        - Увеличение объема данных: Создание или сбор дополнительных данных, которые подчеркивают долгосрочные зависимости, может помочь сети лучше их изучить.

4. Сложность обучения

	- Описание: РНС могут быть сложными для обучения, особенно если они имеют глубокую архитектуру или длинные последовательности.
	- Решение:
        - Упрощение архитектуры: Использование менее сложных моделей или уменьшение количества слоев может помочь.
        - Использование предварительно обученных моделей: Применение трансферного обучения с использованием заранее обученных моделей может ускорить процесс обучения.

5. Затраты на вычисления

	- Описание: Обучение РНС может быть вычислительно затратным, особенно для длинных последовательностей.
	- Решение:
        - Использование GPU: Обучение на графических процессорах может значительно ускорить процесс.
        - Пакетная обработка: Обработка данных партиями (batch processing) для более эффективного использования ресурсов.

#### 3. Какая роль у рекуррентного слоя в рекуррентной нейронной сети?

Рекуррентный слой в рекуррентной нейронной сети (РНС) играет ключевую роль в обработке последовательных данных. Вот основные функции и характеристики рекуррентного слоя:

1. Обработка последовательной информации

	- Рекуррентный слой способен принимать последовательности данных (например, текст, временные ряды, аудио) и обрабатывать их поэлементно, сохраняя информацию о предыдущих состояниях. Это позволяет сети учитывать контекст и взаимосвязи между элементами последовательности.

2. Сохранение состояния

	- Рекуррентные слои используют скрытое состояние (hidden state), которое обновляется на каждом временном шаге. Это состояние хранит информацию о предыдущих входах, позволяя модели “помнить” о контексте, что критично для задач, где порядок данных имеет значение.

3. Циклические соединения

	- В отличие от обычных слоев, рекуррентные слои имеют циклические соединения, которые позволяют передавать информацию не только вперед, но и назад. Это создает возможность для передачи контекста из предыдущих временных шагов на текущий шаг.

4. Гибкость в длине входа

	- Рекуррентные слои могут обрабатывать последовательности переменной длины, что делает их подходящими для различных типов данных, таких как предложения с разным количеством слов или временные ряды с разным количеством наблюдений.

5. Изучение временных зависимостей

	- Рекуррентные слои способны выявлять и изучать временные зависимости, что позволяет эффективно решать задачи, такие как прогнозирование временных рядов, анализ текстов и распознавание речи.

Применение

Рекуррентные слои могут использоваться в различных архитектурах, таких как:

- LSTM (Long Short-Term Memory): Расширенный рекуррентный слой, который помогает преодолевать проблемы исчезающего градиента и сохранять долгосрочные зависимости.
- GRU (Gated Recurrent Units): Более простая альтернатива LSTM, которая также решает проблемы с долгосрочными зависимостями, но с меньшим числом параметров.

#### 4. Какие метрики можно использовать для оценки результатов прогнозирования временных рядов?

Для оценки результатов прогнозирования временных рядов можно использовать различные метрики. Вот некоторые из наиболее распространенных:

1. Средняя абсолютная ошибка (MAE)

	- Формула: ￼$  \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2  $
	- Описание: Измеряет среднюю абсолютную разницу между фактическими значениями и прогнозами. MAE легче интерпретировать, так как выражается в тех же единицах, что и данные.

2. Средняя квадратичная ошибка (MSE)

	- Формула: ￼$  \text{MSE} = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2  $
	- Описание: Измеряет среднюю квадратичную разницу между фактическими значениями и прогнозами. Меньшие значения MSE указывают на лучшее качество модели, но более чувствительна к выбросам.

3. Корень средней квадратичной ошибки (RMSE)

	- Формула: $  \text{RMSE} = \sqrt{\text{MSE}}  $
	- Описание: Это квадратный корень из MSE. RMSE выражается в тех же единицах, что и данные, и позволяет легче интерпретировать ошибки.

4. Средняя абсолютная процентная ошибка (MAPE)

	- Формула: ￼ $ R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}i)^2}{\sum{i=1}^{n} (y_i - \bar{y})^2} $
	- Описание: Измеряет среднюю абсолютную ошибку в процентах. Полезна для оценки производительности модели, особенно когда значения данных варьируются.

5. R-квадрат (R²)

	- Формула: ￼￼$ R^2 = 1 - \frac{\sum_{i=1}^{n} (y_i - \hat{y}i)^2}{\sum{i=1}^{n} (y_i - \bar{y})^2} $
	- Описание: Указывает на долю вариации в зависимой переменной, объясненную моделью. Значения R² находятся в диапазоне от 0 до 1, где 1 означает идеальное соответствие.

6. Долговременные метрики

	- Показатели устойчивости модели: Такие как Mean Absolute Scaled Error (MASE), которые учитывают производительность модели по сравнению с простой моделью, предсказывающей среднее значение временного ряда.

7. Графическое представление

	- Визуализация остаточных ошибок на графиках может помочь в выявлении паттернов, которые могут быть неявными в числовых метриках.